# Predicción de derrame cerebral
### Objetivo
Predecir para una determinada persona, en base a determinadas características fisiológicas y sociales, la posibilidad de que tenga un derrame cerebral. 

In [95]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score

Cargar el dataset "Stroke Prediction". Se encuentra disponible el script `download_dataset.sh` para su descarga desde Kaggle.

In [ ]:
df = pd.read_csv("healthcare-dataset-stroke-data.csv")
df.set_index("id", inplace=True)
df

### Exploración de los datos

In [ ]:
df.stroke.value_counts()

 #### Conclusiones:
 Existe una clara desproporción entre las observaciones con y sin derrame cerebral.

### Imputación de datos faltantes
Se que la razón por la cual faltan datos es completamente aleatoria (MCAR).

In [ ]:
df[df.isnull().any(axis=1)] # the only column with missing values is bmi

In [ ]:
df_no_nan = df.dropna(axis="index", how="any")

fig, axs = plt.subplots(nrows=1, ncols=2)
axs[0].set_title("BMI NO NAN")
sns.countplot(data=df_no_nan, x="stroke", ax=axs[0])
axs[1].set_title("BMI NAN")
sns.countplot(data=df[df.isnull().any(axis=1)], x="stroke", ax=axs[1])

Por el momento, se eliminan las filas con valores faltantes de bmi. Si en el modelo no se utiliza la variable bmi, es importante no eliminar los datos.

In [78]:
df.dropna(axis="index", how="any", inplace=True)

### Transformación de los datos
Transformación de variables categóricas. Posible mediante **One-Hot encoding** dada que la cantidad de categorías es relativamente baja y con poca probabilidad de cambiar/extenderse en el futuro.

In [ ]:
# Cathegoric variables transformation
print("Applying One-Hot encoding to:")
for label, to_drop in [
    ("gender", "Other"),
    ("ever_married", "No"),
    ("work_type", "children"),
    ("Residence_type", "Rural"),
    ("smoking_status", "Unknown"),
]:
    unique_values = df[label].unique()
    print(label + ":", unique_values)
    prefix = "is" if len(unique_values) > 2 else label
    one_hot = pd.get_dummies(data=df[label], prefix=prefix).drop(
        prefix + "_" + to_drop if prefix else to_drop, axis=1
    )
    df.drop(label, axis=1, inplace=True)
    df = df.join(one_hot)
df.head()

### Selección de los datos
Eliminación de variables poco útiles para reducir la dimensión.

In [ ]:
df.describe()

### Modelo de regresión logística
División entre set de entrenamiento y set de validación

In [83]:
X = df.drop("stroke", axis=1).values
y = df["stroke"].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=0, stratify=y
)

Escalamiento de los datos

In [85]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Entrenamiento del modelo

In [ ]:
model_logistic = LogisticRegression(random_state = 0, class_weight="balanced")
model_logistic.fit(X_train, y_train)

Predicción con el set de validación

In [87]:
y_pred = model_logistic.predict(X_test)

Análisis de resultados para un modelo de regresión logística

In [ ]:
print("F1-score:", f1_score(y_true=y_test, y_pred=y_pred))
cm_logistic = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm_logistic,
                              display_labels=model_logistic.classes_)

fig, ax = plt.subplots(figsize=(6,6))
ax.grid(False)
disp.plot(ax=ax)
ax.set_title("Clasificador Regresión Logística");

### Clasificador KNN

In [97]:
classifier_knn = KNeighborsClassifier(n_neighbors=23, metric="minkowski", p=2)
classifier_knn.fit(X_train, y_train)
y_pred  = classifier_knn.predict(X_test)

In [ ]:
print("F1-score:", f1_score(y_true=y_test, y_pred=y_pred))
cm_logistic = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm_logistic,
                              display_labels=model_logistic.classes_)

fig, ax = plt.subplots(figsize=(6,6))
ax.grid(False)
disp.plot(ax=ax)
ax.set_title("Clasificador KNN");